In the initial step of the Least Mean Squares (LMS) adaptation process for the interference canceling filter, we need to define and compute several signals and variables. Let's present each of them:

1. **Received Signal**: $ y[k] $ (Dimension: $ m \times 1 $)
2. **Transmitted Symbol Sequence**: $ a[k] $ (Dimension: $ m \times 1 $)
3. **Additive Noise Sequence**: $ v[k] $ (Dimension: $ m \times 1 $)
4. **Desired Response Signal**: $ d[k] $ (Dimension: $ 1 \times 1 $)
5. **Error Signal**: $ \epsilon[k] $ (Dimension: $ 1 \times 1 $)
6. **Input Signal**: $ x[k] $ (Dimension: $ m \times 1 $)
7. **Interference Canceling Filter**: $ \mathbf{f} $ (Dimension: $ 1 \times m $)
8. **Orthogonal Complement Filter**: $ \mathbf{h}^{\perp H} $ (Dimension: $ m \times m $)
9. **Received Power**: $ \sigma_a^2 $
10. **Covariance Matrix of Noise**: $ R_{\mathbf{VV}} $

Here's how we can describe each of these signals in the context of the initial step:

1. **Received Signal**: $ y[k] $ is the received signal at time $ k $, containing $ m $ subchannels.
2. **Transmitted Symbol Sequence**: $ a[k] $ is the transmitted symbol sequence at time $ k $, also containing $ m $ elements.
3. **Additive Noise Sequence**: $ v[k] $ is the additive noise sequence at time $ k $, containing $ m $ elements.
4. **Desired Response Signal**: $ d[k] $ is the desired response signal at time $ k $, which is the output of the spatial matched filter $ \mathbf{y}_1[k] $.
5. **Error Signal**: $ \epsilon[k] $ is the error signal at time $ k $, representing the difference between the desired response signal $ d[k] $ and the estimated response obtained by filtering the input signal $ x[k] $ with the interference canceling filter $ \mathbf{f} $.
6. **Input Signal**: $ x[k] $ is the input signal at time $ k $, which is also the output $ \mathbf{y}_2[k] $ of the orthogonal complement filter $ \mathbf{h}^{\perp H} $.
7. **Interference Canceling Filter**: $ \mathbf{f} $ is the interference canceling filter, represented as a row vector of length $ m $.
8. **Orthogonal Complement Filter**: $ \mathbf{h}^{\perp H} $ is the orthogonal complement filter, which complements the interference canceling filter.
9. **Received Power**: $ \sigma_a^2 $ represents the transmitted power.
10. **Covariance Matrix of Noise**: $ R_{\mathbf{VV}} $ is the covariance matrix of the noise, which is spatially colored.

These signals and variables are essential for understanding and implementing the LMS adaptation process for the interference canceling filter.

In [1]:
using FFTW
using LinearAlgebra
using Random
using Plots
using Statistics

In [2]:
include("_support/operations.jl")

norm (generic function with 15 methods)

In [3]:
abstract type AbstractSignal{T} end

struct FilterParams{T} <: AbstractSignal{T}
    h::Vector{T}   # Channel Vector
    h₊ᴴ::Matrix{T} # Orthogonal complement filter h_perp_H
    σ::T         # Sigma
end

In [ ]:
# Define the dimension of the channel vector
n = 10

# Seed for reproducibility
Random.seed!(123)

# Generate a random complex channel vector
h = randn(ComplexF64, n)

# Generate a random vector for the orthogonal complement
hꜜ = randn(ComplexF64, n)
while hꜜ ⋅ h ≈ 0
    hꜜ = randn(ComplexF64, n)
end

println("Channel vector h:", h)
println("Vector in the orthogonal complement:", hꜜ)

In [ ]:
# Extract real and imaginary parts for plotting
hᵣ  = real.(h)
hᵢ  = imag.(h)
hꜜᵣ = real.(hꜜ)
hꜜᵢ = imag.(hꜜ);

In [ ]:
# Plot the vectors on the complex plane
plot(hᵣ, hᵢ
    , seriestype=:scatter
    , label="Channel vector h"
    , xlabel="Real part", ylabel="Imaginary part"
)
plot!(hꜜᵣ, hꜜᵢ
    , seriestype=:scatter
    , label="Vector in orthogonal complement"
)

In [ ]:
j = √Complex(-1)

In [ ]:
using LaTeXStrings

In [ ]:
# Define Z coordinates based on the magnitudes of real and imaginary parts
hₔ = abs.(hᵣ  + j*hᵢ)
hꜜₔ = abs.(hꜜᵣ + j*hꜜᵢ)

# Plot the vectors in 3D
plot(hᵣ, hᵢ, hₔ
    , seriestype=:scatter3d
    , label="Channel vector h", legend=:topright
    , xlabel=L"$\scr{Re}$", ylabel=L"$\scr{Im}$", zlabel="Magnitude"
)
plot!(hꜜᵣ, hꜜᵢ, hꜜₔ
    , seriestype=:scatter3d
    , label="Vector in orthogonal complement"
)

In [ ]:
𝖤 = mean

In [ ]:
v = randn(ComplexF64, 10) # Generate a signal

# Compute autocorrelation matrix R
# 𝑅ᵥᵥ = autocorrelation_matrix(x[1:N])
N = length(v)
𝑅ᵥᵥ = [i != j ? 
        𝖤( v[ 1 : N - |(i-j) ] .* v[ |(i-j) + 1 : N] ) : # |() is the abs operator 
        𝖤(v .* v) # 𝖤 is the mean operator - Expected Value
            for i in 1:N, j in 1:N
        ];

In [ ]:
# LMMSE Filter f^o
fᵒ = ((h)ᴴ * 𝑅ᵥᵥ * hꜜ) * ((hꜜ)ᴴ * 𝑅ᵥᵥ * hꜜ)⁻¹

# References

In [ ]:
# Real matrix
A_real = [1 2; 3 4]

# Hermitian transpose of a real matrix (equivalent to transpose)
A_real_transpose = transpose(A_real)
# or
A_real_transpose = A_real'

# Complex matrix
A_complex = [1+2im 3+4im; 5+6im 7+8im]

# Hermitian transpose of a complex matrix
A_complex_herm_transpose = adjoint(A_complex)
# or
A_complex_herm_transpose = transpose(A_complex)